In [ ]:
%matplotlib inline
import os
import re
import sys
from oqmbt.oqt_project import OQtProject
from oqmbt.notebooks.sources_shallow_fault.create_fault_sources_from_geojson import get_fault_sources
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.geo.surface import SimpleFaultSurface
from openquake.hazardlib.mfd import EvenlyDiscretizedMFD
from openquake.hazardlib.scalerel.wc1994 import WC1994
from openquake.hazardlib.sourcewriter import write_source_model

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]

## Parameters

In [ ]:
slip_rate_class = 'suggested' 
m_low = model.faults_lower_threshold_magnitude
bin_width = model.mfd_binwidth
scalerel_name = model.msr
rupture_mesh_spacing = model.fault_rupture_mesh_spacing
upper_seismogenic_depth = model.upper_seismogenic_depth
## THIS PARAMETER IS NOT USED BUT THIS PART NEEDS TO BE IMPROVED
lower_seismogenic_depth=10.0 
rupture_aspect_ratio=2.0
temporal_occurrence_model=PoissonTOM(1.0)
aseismic_coeff=0.9
# this is the geojson file
filename = os.path.abspath(os.path.join(oqtkp.directory, model.fault_geojson_filename))

## Set the magnitude scaling relationship
The options available for the magnitude scaling relationship are:
* `scalrel_name = 'WC1994`' <br> Wells and Coppersmith (1994) scaling relationship
* '`PointMSR`' Scaling relationship to be used for the creation of tiny ruptures resembling points
* '`CEUS2011`' Scaling relationship developed within the CEUS SSC project

In [ ]:
import importlib
module = importlib.import_module('openquake.hazardlib.scalerel')
my_class = getattr(module, scalerel_name)
msr = my_class()

## Creating the xml file

In [ ]:
b_gr = 1.0 # this is a dummy value that will be modified during following steps
sources = get_fault_sources(filename, 
                            slip_rate_class, 
                            bin_width, 
                            m_low, 
                            b_gr,
                            rupture_mesh_spacing, 
                            upper_seismogenic_depth,
                            lower_seismogenic_depth, 
                            msr,
                            rupture_aspect_ratio, 
                            temporal_occurrence_model,
                            aseismic_coeff)

In [ ]:
# write the final fault model
dest = model.faults_xml_filename
if not os.path.exists(os.path.dirname(dest)):
    print(os.path.dirname(dest))
    os.mkdir(os.path.dirname(dest))
write_source_model(dest, sources, 'Test')